In [1]:
%pip install playwright

!playwright install

!playwright install-deps

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install requests beautifulsoup4 tqdm lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install patool

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install playwright playwright-stealth
!playwright install chromium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import time
import os
import json
import traceback
import threading
import sys
import asyncio 

from playwright.sync_api import sync_playwright

def classify_topic(title, query):
    text = f"{title} {query}".lower()
    if any(kw in text for kw in ["nông nghiệp", "thủy sản", "lâm nghiệp", "cây trồng", "vật nuôi", "giống", "thổ nhưỡng", "phân bón"]):
        return "Nông nghiệp"
    elif any(kw in text for kw in ["trái đất", "địa chất", "khí hậu", "thủy văn", "khí tượng", "địa lý", "khoáng sản", "môi trường", "tai biến"]):
        return "Khoa học Trái đất"
    elif any(kw in text for kw in ["giao thông", "đường bộ", "bê tông nhựa", "mặt đường", "lưu lượng", "tai nạn", "cầu đường", "vận tải", "đường cao tốc", "xe buýt", "nút giao"]):
        return "Kỹ thuật Xây dựng và Giao thông"
    else:
        if query in ["nông nghiệp", "thủy sản", "lâm nghiệp", "cây trồng", "vật nuôi"]:
            return "Nông nghiệp"
        elif query in ["giao thông", "đường bộ", "bê tông nhựa", "mặt đường", "lưu lượng", "tai nạn", "cầu đường", "vận tải", "đường cao tốc", "xe buýt", "nút giao"]:
            return "Kỹ thuật Xây dựng và Giao thông"
        else:
            return "Khoa học Trái đất"

def crawl_vjst_direct_pdf(target_count=1000):
    output_file = "vjst_data_final.json"
    data_list = []
    
    # Load lại dữ liệu để chạy tiếp nếu bị ngắt
    if os.path.exists(output_file):
        try:
            with open(output_file, "r", encoding="utf-8") as f:
                data_list = json.load(f)
            print(f"[*] Tìm thấy file cũ, tiếp tục từ bài thứ {len(data_list)}...", flush=True)
        except:
            pass

    search_queries = [
        "nông nghiệp", "thủy sản", "lâm nghiệp", "cây trồng", "vật nuôi",
        "khoa học trái đất", "địa chất", "biến đổi khí hậu", "thủy văn", "khí tượng", "giao thông", "đường bộ", 
        "mặt đường", "tai nạn", "vận tải", "đường cao tốc"
    ]

    with sync_playwright() as p:
        print("[*] Đang khởi động trình duyệt...", flush=True)
        browser = p.chromium.launch(headless=False, args=["--no-sandbox", "--disable-dev-shm-usage"])
        context = browser.new_context()
        main_page = context.new_page()

        print(f"[*] Mục tiêu: {target_count} bài (ĐỊNH DẠNG CHUẨN KÈM LINK PDF). Bắt đầu crawl...", flush=True)

        for query in search_queries:
            if len(data_list) >= target_count: break
                
            print(f"\n==========================================")
            print(f"[*] ĐANG QUÉT TỪ KHÓA: {query.upper()}")
            print(f"==========================================", flush=True)
            
            for page_num in range(1, 51): 
                if len(data_list) >= target_count: break

                current_url = f"https://b.vjst.vn/index.php/ban_b/search/search?query={query}&searchPage={page_num}"
                print(f"--- Đang tải trang {page_num} | Tổng: {len(data_list)}/{target_count} ---", flush=True)

                try:
                    time.sleep(1.5)
                    main_page.goto(current_url, timeout=60000)
                    
                    try:
                        main_page.wait_for_selector("div.obj_article_summary", timeout=10000)
                    except:
                        print("   -> Không còn bài báo nào. Chuyển từ khóa...", flush=True)
                        break 
                    
                    articles = main_page.locator("div.obj_article_summary")
                    count = articles.count()
                    if count == 0: break 
                        
                    for i in range(count):
                        if len(data_list) >= target_count: break
                        
                        article = articles.nth(i)
                        
                        try:
                            link_loc = article.locator(".title a, h2 a, h3 a").first
                            if link_loc.count() == 0: continue
                                
                            title = link_loc.inner_text(timeout=2000).strip()
                            article_url = link_loc.get_attribute("href", timeout=2000)

                            if not article_url or "/article/view/" not in article_url:
                                continue
                            
                            # Kiểm tra trùng lặp title trước để đỡ tốn công mở tab
                            if any(d['title'] == title for d in data_list):
                                continue

                            # --- CHIẾN THUẬT TAB ẨN ĐỂ LẤY LINK PDF ---
                            pdf_link = None
                            detail_page = context.new_page() # Mở 1 tab con
                            try:
                                detail_page.goto(article_url, timeout=30000)
                                pdf_btn = detail_page.locator("a.obj_galley_link.pdf, a.galley-link.pdf").first
                                
                                if pdf_btn.count() > 0:
                                    galley_url = pdf_btn.get_attribute("href")
                                    # Mẹo biến link view thành link download trực tiếp
                                    pdf_link = galley_url.replace("/view/", "/download/")
                            except Exception:
                                pass
                            finally:
                                detail_page.close() # Xong việc đóng tab con lại

                            if not pdf_link:
                                continue # Không có PDF thì bỏ qua bài này

                            topic = classify_topic(title, query)
                            
                            # --- ĐỊNH DẠNG CHUẨN YÊU CẦU ---
                            record = {
                                "source": "Tạp chí Khoa học và Công nghệ Việt Nam (VJST)",
                                "topic": topic,
                                "title": title,
                                "pdf_url": pdf_link
                            }
                            
                            data_list.append(record)
                            print(f"   + [{topic}] {len(data_list)}: {title[:50]}...", flush=True)
                            
                            # Lưu json liên tục
                            with open(output_file, "w", encoding="utf-8") as f:
                                json.dump(data_list, f, ensure_ascii=False, indent=4)

                        except Exception:
                            continue

                except Exception as e:
                    if "Target page, context or browser has been closed" in str(e):
                        print("\n[!] ĐÃ NHẬN LỆNH ĐÓNG TRÌNH DUYỆT TỪ NGƯỜI DÙNG. Đang lưu và thoát...", flush=True)
                        return
                    break 

        try:
            browser.close()
        except:
            pass
        print(f"\n[HOÀN THÀNH] Đã thu thập xong {len(data_list)} bài.", flush=True)

# --- KHỞI TẠO LUỒNG ĐỘC LẬP ---
def run_in_thread():
    if sys.platform == 'win32':
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    try:
        crawl_vjst_direct_pdf(1000)
    except Exception as e:
        traceback.print_exc()

print("[*] Đang khởi động Crawler bản hoàn chỉnh...", flush=True)
thread = threading.Thread(target=run_in_thread)
thread.start()
thread.join()

[*] Đang khởi động Crawler bản hoàn chỉnh...
[*] Tìm thấy file cũ, tiếp tục từ bài thứ 729...
[*] Đang khởi động trình duyệt...
[*] Mục tiêu: 1000 bài (ĐỊNH DẠNG CHUẨN KÈM LINK PDF). Bắt đầu crawl...

[*] ĐANG QUÉT TỪ KHÓA: NÔNG NGHIỆP
--- Đang tải trang 1 | Tổng: 729/1000 ---
--- Đang tải trang 2 | Tổng: 729/1000 ---
--- Đang tải trang 3 | Tổng: 729/1000 ---
--- Đang tải trang 4 | Tổng: 729/1000 ---
--- Đang tải trang 5 | Tổng: 729/1000 ---
--- Đang tải trang 6 | Tổng: 729/1000 ---
--- Đang tải trang 7 | Tổng: 729/1000 ---
--- Đang tải trang 8 | Tổng: 729/1000 ---
--- Đang tải trang 9 | Tổng: 729/1000 ---
--- Đang tải trang 10 | Tổng: 729/1000 ---
--- Đang tải trang 11 | Tổng: 729/1000 ---
   -> Không còn bài báo nào. Chuyển từ khóa...

[*] ĐANG QUÉT TỪ KHÓA: THỦY SẢN
--- Đang tải trang 1 | Tổng: 729/1000 ---
--- Đang tải trang 2 | Tổng: 729/1000 ---
--- Đang tải trang 3 | Tổng: 729/1000 ---
--- Đang tải trang 4 | Tổng: 729/1000 ---
--- Đang tải trang 5 | Tổng: 729/1000 ---
--- Đang t